# Facebook & Insta ANOVA

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 200)

In [3]:
fpath = os.path.join('data')
df_sns_clean = pd.read_csv(os.path.join(fpath,'df_sns_cleaned.csv'),encoding = 'utf-8')


### 데이터 확인

In [4]:
df_sns_clean.head()

,BrandID,BrandCategory,PhotoID,PostURL,PhotoURL,PostDate,PostTime,DownloadDate,Type,type_video,Like,LikeAlbum,Comments,Comments_album,NumHashtags,Content,ContentWithoutLink,1SNS,2brand,3model,31number,32face,33body,34modelposition,4background,5text,51words,52,53textposition,54promotingmessa,6brandname,61brandexpose,62,63brandposition,64brandvisib,65partofprod,66whproduct,7logo,71logoexposure,72,73logoposition,74logotype,75logovisib,76logopartofprod,8product,81productexpose,82,83productposition,9,10angle,11,12hashtag,12_1number,12_2_1content,12_2_2,12_2_3,12_2_4,12_2_5,12_2_6,13text,13_1,13_2promo,13_2promo_wo_kor,13_3link,BrandCategory_SPA,BrandCategory_casual,BrandCategory_luxuries,BrandCategory_sport,34modelposition_1,34modelposition_2,34modelposition_3,34modelposition_4,34modelposition_5,34modelposition_6,34modelposition_7,34modelposition_8,34modelposition_9,4background_indoor,4background_others,4background_outdoor,4background_wall,53textposition_1,53textposition_2,53textposition_3,53textposition_4,53textposition_5,53textposition_6,53textposition_7,53textposition_8,53textposition_9,63brandposition_1,63brandposition_2,63brandposition_3,63brandposition_4,63brandposition_5,63brandposition_6,63brandposition_7,63brandposition_8,63brandposition_9,66whproduct_accessories,66whproduct_bag,66whproduct_clothes_bottom,66whproduct_clothes_top,66whproduct_cosmetics,66whproduct_hat,66whproduct_others,66whproduct_shoes,73logoposition_1,73logoposition_2,73logoposition_3,73logoposition_4,73logoposition_5,73logoposition_6,73logoposition_7,73logoposition_8,73logoposition_9,74logotype_both,74logotype_image,74logotype_text,83productposition_1,83productposition_2,83productposition_3,83productposition_4,83productposition_5,83productposition_6,83productposition_7,83productposition_8,83productposition_9,10angle_dutch,10angle_extreme_high,10angle_eye,10angle_high,10angle_low,10angle_other
0,1,SPA,1,https://www.facebook.com/hmtheus/videos/202686...,https://m.facebook.com/story.php?story_fbid=20...,2018-01-08,2:24,2018-05-26,VIDEO,1,187,NaN,134.0,NaN,3.0,Holographic highlighters and icy-cool shimmer ...,Holographic highlighters and icy-cool shimmer ...,1,1,1,1.0,1.0,2.0,"1,2,3,4,5,6,8,9",1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,NaN,"6,8,9",NaN,3.0,NaN,1,3.0,2.0,2.0,1.0,0.0,0.0,0.0,1,25,2.0,0.0,1.0,1,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0,1,0,0,0
1,1,SPA,2,https://www.facebook.com/hmtheus/photos/a.1514...,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...,2018-01-07,7:32,2018-05-26,PICTURE,0,434,NaN,100.0,NaN,1.0,Somewhere between versatile neutral and a stat...,Somewhere between versatile neutral and a stat...,1,1,0,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,1,1.0,NaN,"2,3",0.0,1.0,"1,2",1,1.0,NaN,"2,3",2.0,0.0,1.0,1,1.0,NaN,"1,2,3,4,5,6,7,8,9",NaN,3.0,NaN,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1,19,1.0,1.0,1.0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0
2,1,SPA,3,https://www.facebook.com/hmtheus/photos/ms.c.e...,https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/...,2018-01-07,1:47,2018-05-26,PICTURE,0,1,148.0,0.0,44.0,2.0,"To get your skin through the cold season, you ...","To get your skin through the cold season, you ...",1,1,1,1.0,1.0,2.0,"1,2,3,4,5,6,7,8,9",1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,NaN,"6,7,9",NaN,3.0,NaN,1,2.0,2.0,2.0,0.0,0.0,0.0,0.0,1,19,2.0,0.0,1.0,1,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0

In [27]:
from scipy import stats
import researchpy as rp
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [13]:
print('%.2f'%1)

1.00


In [17]:
formula = "%s ~ C(%s) + C(%s) + C(%s):C(%s)"%(dv,dv1,dv2,dv1,dv2)
formula

'a'

In [52]:
def anova_sns(df,dv1,dv2,iv):
    data = df.copy()
    data[(df[iv].isna() == False) & (df[dv1].isna() == False) & (df[dv2].isna() == False)]
    formula = "%s ~ C(%s) + C(%s) + C(%s):C(%s)"%(iv,dv1,dv2,dv1,dv2)
    lm = ols(formula,df).fit()
    
    
    return anova_lm(lm,typ=2)

In [53]:
dv_sns = 'type_video'
iv = 'Like'
dv = 'Type'
result = anova_sns(df_sns_clean,dv_sns,dv,iv)

ValueError: shapes (1993,1490) and (1993,1490) not aligned: 1490 (dim 1) != 1993 (dim 0)

In [48]:
anova_lm(result)

ValueError: shape mismatch: value array of shape (3,1491) could not be broadcast to indexing result of shape (3,2)

In [ ]:
data = df.copy()
dv_sns = '1SNS'
iv = 'Like'
dv = '3model'
formula = "%s ~ C(%s) + C(%s) + C(%s):C(%s)"%(dv,dv1,dv2,dv1,dv2)
lm = ols(formula,df).fit

In [23]:
result

'3model ~ C(1SNS) + C(3model) + C(1SNS):C(3model)'

### 3Model

In [31]:
col = '3model'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)


df_model = df_sns_clean[df_sns_clean['3model'] == 1]
chi_col = ['32face', '33body']
for col in chi_col:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    p = result[1].iloc[1,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)
    
    
#result[1].iloc[1,1]

odel]
          3model            
               N     Y   All
1SNS                        
Facebook     312   688  1000
instagram    337   660   997
All          649  1348  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.5401
1                    p-value =    0.2146
2               Cramer's phi =    0.0278 

Importance: X

[32face]
          32face            
               N     Y   All
1SNS                        
Facebook     140   548   688
instagram    150   510   660
All          290  1058  1348 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.1286
1                    p-value =    0.2881
2               Cramer's phi =    0.0289 

Importance: X

[33body]
          33body           
             2.0    Y   All
1SNS                       
Facebook     395  293   688
instagram    358  301   659
All          753  594  1347 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.3021
1         

In [11]:
result = t_test_sns(df_model,'31number')
p = result[1].iloc[3,1]
score = importance(p)

print('[31number]')
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[31number]
    Variable       N      Mean        SD        SE  95% Conf.  Interval
0   Facebook   688.0  1.861919  2.848908  0.108614   1.648664  2.075173
1  Instagram   660.0  2.022727  3.016057  0.117400   1.792204  2.253250
2   combined  1348.0  1.940653  2.931949  0.079857   1.783996  2.097310 

                                Independent t-test    results
0             Difference (Facebook - Instagram) =     -0.1608
1                            Degrees of freedom =   1346.0000
2                                             t =     -1.0066
3                         Two side test p value =      0.3143
4  Mean of Facebook > mean of Instagram p value =      0.1571
5  Mean of Facebook < mean of Instagram p value =      0.8429
6                                     Cohen's d =     -0.0548
7                                     Hedge's g =     -0.0548
8                                 Glass's delta =     -0.0564
9                                             r =      0.0274 

Importance: X



In [32]:
position_col_name = '34modelposition_'
df_position = df_model

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    print(result[0],'\n')
    print(result[1],'\n')
    
    p = result[1].iloc[1,1]
    score = importance(p)
    print('Importance: %s\n'%score)

[odelposition]]


odelposition_1]
          34modelposition_1           
                          N    Y   All
1SNS                                  
Facebook                306  382   688
instagram               288  372   660
All                     594  754  1348 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.0965
1                    p-value =    0.7560
2               Cramer's phi =    0.0085 

Importance: X

odelposition_2]
          34modelposition_2            
                          N     Y   All
1SNS                                   
Facebook                 78   610   688
instagram                59   601   660
All                     137  1211  1348 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.1212
1                    p-value =    0.1453
2               Cramer's phi =    0.0397 

Importance: X

odelposition_3]
          34modelposition_3           
                          N    Y   All
1SNS            

### 4background

In [14]:
chi_iter_sns(df_sns_clean,'4background',{1:'wall',2:'indoor',3:'outdoor',4:'others'})

[indoor]
          indoor           
               0    1   All
1SNS                       
Facebook     824  176  1000
instagram    880  117   997
All         1704  293  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   13.7164
1                    p-value =    0.0002
2               Cramer's phi =    0.0829 

Importance: ***

[others]
          others          
               0   1   All
1SNS                      
Facebook     944  56  1000
instagram    956  41   997
All         1900  97  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.3909
1                    p-value =    0.1220
2               Cramer's phi =    0.0346 

Importance: X

[outdoor]
          outdoor           
                0    1   All
1SNS                        
Facebook      675  325  1000
instagram     765  232   997
All          1440  557  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   21.1484
1               

### 5Text

In [15]:
col = '5text'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[5text]
          5text           
              N    Y   All
1SNS                      
Facebook    839  161  1000
instagram   884  112   996
All        1723  273  1996 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    9.9622
1                    p-value =    0.0016
2               Cramer's phi =    0.0706 

Importance: **



In [18]:
df_text = df_sns_clean[df_sns_clean['5text'] == 1]

In [19]:

#result = t_test_sns(df_test,'31number')


ttest_col = ['51words']
for col in ttest_col:
    print('[%s]'%col)
    result = t_test_sns(df_text,col)
    p = result[1].iloc[3,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)
    
    

[51words]
    Variable      N      Mean         SD        SE  95% Conf.  Interval
0   Facebook  161.0  6.416149   8.595463  0.677417   5.078316  7.753982
1  Instagram  112.0  6.160714  13.061568  1.234202   3.715060  8.606368
2   combined  273.0  6.311355  10.634733  0.643643   5.044199  7.578511 

                                Independent t-test   results
0             Difference (Facebook - Instagram) =     0.2554
1                            Degrees of freedom =   271.0000
2                                             t =     0.1949
3                         Two side test p value =     0.8456
4  Mean of Facebook > mean of Instagram p value =     0.5772
5  Mean of Facebook < mean of Instagram p value =     0.4228
6                                     Cohen's d =     0.0240
7                                     Hedge's g =     0.0239
8                                 Glass's delta =     0.0297
9                                             r =     0.0118 

Importance: X



In [20]:
col = '54promotingmessa'

result = chi_sns(df_text,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[54promotingmessa]
          54promotingmessa         
                         N   Y  All
1SNS                               
Facebook               141  20  161
instagram              103   9  112
All                    244  29  273 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.3387
1                    p-value =    0.2473
2               Cramer's phi =    0.0700 

Importance: X



In [33]:
position_col_name = '53textposition_'
df_position = df_text

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    print(result[0],'\n')
    print(result[1],'\n')
    p = result[1].iloc[1,1]
    score = importance(p)
    print('Importance: %s\n'%score)

[[53textposition]]


[53textposition_1]
          53textposition_1         
                         N   Y  All
1SNS                               
Facebook                76  17   93
instagram               40  11   51
All                    116  28  144 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.2275
1                    p-value =    0.6334
2               Cramer's phi =    0.0397 

Importance: X

[53textposition_2]
          53textposition_2         
                         N   Y  All
1SNS                               
Facebook                72  21   93
instagram               32  19   51
All                    104  40  144 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    3.5354
1                    p-value =    0.0601
2               Cramer's phi =    0.1567 

Importance: X

[53textposition_3]
          53textposition_3         
                         N   Y  All
1SNS                               
Facebook        

### 6Brandname

In [22]:
col = '6brandname'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[6brandname]
          6brandname           
                   N    Y   All
1SNS                           
Facebook         812  188  1000
instagram        811  186   997
All             1623  374  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.0068
1                    p-value =    0.9343
2               Cramer's phi =    0.0018 

Importance: X



In [34]:
df_brand = df_sns_clean[(df_sns_clean['6brandname'] == 1)]

cols = ['61brandexpose','65partofprod']
for col in cols: 
    result = chi_sns(df_brand,col)
    p = result[1].iloc[1,1]
    score = importance(p)

    print('[%s]'%col)
    print(result[0],'\n')
    print(result[1],'\n')
    print('Importance: %s\n'%score)
    
col = '64brandvisib'
result = chi_sns(df_brand,col,{0:'low',1:'high'})
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')

print('Importance: %s\n'%score)

[61brandexpose]
          61brandexpose          
                      N    Y  All
1SNS                             
Facebook             11  177  188
instagram            25  161  186
All                  36  338  374 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    6.1913
1                    p-value =    0.0128
2               Cramer's phi =    0.1287 

Importance: *

[65partofprod]
          65partofprod          
                     N    Y  All
1SNS                            
Facebook            91   97  188
instagram           34  152  186
All                125  249  374 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   38.1310
1                    p-value =    0.0000
2               Cramer's phi =    0.3193 

Importance: ***

[64brandvisib]
          64brandvisib          
                  high  low  All
1SNS                            
Facebook            94   94  188
instagram           95   91  186
All                

In [35]:

position_col_name = '63brandposition_'
df_position = df_brand

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    p = result[1].iloc[1,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)

[[63brandposition]]


[63brandposition_1]
          63brandposition_1         
                          N   Y  All
1SNS                                
Facebook                 70   8   78
instagram                58  11   69
All                     128  19  147 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.0516
1                    p-value =    0.3051
2               Cramer's phi =    0.0846 

Importance: X

[63brandposition_2]
          63brandposition_2         
                          N   Y  All
1SNS                                
Facebook                 67  11   78
instagram                55  14   69
All                     122  25  147 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.9930
1                    p-value =    0.3190
2               Cramer's phi =    0.0822 

Importance: X

[63brandposition_3]
          63brandposition_3         
                          N   Y  All
1SNS                              

In [55]:

position_col_name = '66whproduct_'
df_position = df_brand

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    print(result[0],'\n')
    print(result[1],'\n')
    p = result[1].iloc[1,1]
    score = importance(p)
    print('Importance: %s\n'%score)

[[66whproduct]]


[66whproduct_accessories]
          66whproduct_accessories       
                                N  Y All
1SNS                                    
Facebook                       43  1  44
instagram                      53  0  53
All                            96  1  97 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.2171
1                    p-value =    0.2699
2               Cramer's phi =    0.1120 

Importance: X

[66whproduct_bag]
          66whproduct_bag       
                        N  Y All
1SNS                            
Facebook               39  5  44
instagram              51  2  53
All                    90  7  97 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.0685
1                    p-value =    0.1504
2               Cramer's phi =    0.1460 

Importance: X

[66whproduct_clothes_bottom]
          66whproduct_clothes_bottom       
                                   N  Y All
1SNS       

### 7Brandlogo

In [27]:
col = '7logo'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[7logo]
          7logo           
              N    Y   All
1SNS                      
Facebook    623  377  1000
instagram   591  406   997
All        1214  783  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.9131
1                    p-value =    0.1666
2               Cramer's phi =    0.0310 

Importance: X



In [28]:
df_logo = df_sns_clean[(df_sns_clean['7logo'] == 1)]

##
col = '71logoexposure'
result = chi_sns(df_logo,col,{1:'whole',0:'part'})
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

##
col = '75logovisib'
result = chi_sns(df_logo,col,{0:'low',1:'high'})
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

##
col = '76logopartofprod'
result = chi_sns(df_logo,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[71logoexposure]
          71logoexposure           
                    part whole  All
1SNS                               
Facebook              14   362  376
instagram             30   376  406
All                   44   738  782 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    4.9401
1                    p-value =    0.0262
2               Cramer's phi =    0.0795 

Importance: *

[75logovisib]
          75logovisib          
                 high  low  All
1SNS                           
Facebook          192  185  377
instagram         247  159  406
All               439  344  783 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    7.7924
1                    p-value =    0.0052
2               Cramer's phi =    0.0998 

Importance: **

[76logopartofprod]
          76logopartofprod          
                         N    Y  All
1SNS                                
Facebook               107  270  377
instagram               46 

In [30]:
chi_iter_sns(df_logo,'74logotype',{1:'Image',2:'Text',3:'Both'})

[Both]
          Both          
             0    1  All
1SNS                    
Facebook   272  105  377
instagram  273  133  406
All        545  238  783 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.2249
1                    p-value =    0.1358
2               Cramer's phi =    0.0533 

Importance: X

[Image]
          Image          
              0    1  All
1SNS                     
Facebook    167  210  377
instagram   173  233  406
All         340  443  783 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.2262
1                    p-value =    0.6343
2               Cramer's phi =    0.0170 

Importance: X

[Text]
          Text          
             0    1  All
1SNS                    
Facebook   315   62  377
instagram  366   40  406
All        681  102  783 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    7.5007
1                    p-value =    0.0062
2               Cramer's phi = 

In [36]:

position_col_name = '73logoposition_'
df_position = df_logo

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    print(result[0],'\n')
    print(result[1],'\n')
    p = result[1].iloc[1,1]
    score = importance(p)
    print('Importance: %s\n'%score)

[[73logoposition]]


[73logoposition_1]
          73logoposition_1         
                         N   Y  All
1SNS                               
Facebook               191  29  220
instagram              194  20  214
All                    385  49  434 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.5938
1                    p-value =    0.2068
2               Cramer's phi =    0.0606 

Importance: X

[73logoposition_2]
          73logoposition_2         
                         N   Y  All
1SNS                               
Facebook               173  47  220
instagram              167  47  214
All                    340  94  434 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.0229
1                    p-value =    0.8796
2               Cramer's phi =    0.0073 

Importance: X

[73logoposition_3]
          73logoposition_3         
                         N   Y  All
1SNS                               
Facebook        

### 8Product Exposure

In [37]:
pd.DataFrame(df_sns_clean[(df_sns_clean['8product'] == 1)].isna().sum()).T

,BrandID,BrandCategory,PhotoID,PostURL,PhotoURL,PostDate,PostTime,DownloadDate,Type,type_video,Like,LikeAlbum,Comments,Comments_album,NumHashtags,Content,ContentWithoutLink,1SNS,2brand,3model,31number,32face,33body,34modelposition,4background,5text,51words,52,53textposition,54promotingmessa,6brandname,61brandexpose,62,63brandposition,64brandvisib,65partofprod,66whproduct,7logo,71logoexposure,72,73logoposition,74logotype,75logovisib,76logopartofprod,8product,81productexpose,82,83productposition,9,10angle,11,12hashtag,12_1number,12_2_1content,12_2_2,12_2_3,12_2_4,12_2_5,12_2_6,13text,13_1,13_2promo,13_2promo_wo_kor,13_3link,BrandCategory_SPA,BrandCategory_casual,BrandCategory_luxuries,BrandCategory_sport,34modelposition_1,34modelposition_2,34modelposition_3,34modelposition_4,34modelposition_5,34modelposition_6,34modelposition_7,34modelposition_8,34modelposition_9,4background_indoor,4background_others,4background_outdoor,4background_wall,53textposition_1,53textposition_2,53textposition_3,53textposition_4,53textposition_5,53textposition_6,53textposition_7,53textposition_8,53textposition_9,63brandposition_1,63brandposition_2,63brandposition_3,63brandposition_4,63brandposition_5,63brandposition_6,63brandposition_7,63brandposition_8,63brandposition_9,66whproduct_accessories,66whproduct_bag,66whproduct_clothes_bottom,66whproduct_clothes_top,66whproduct_cosmetics,66whproduct_hat,66whproduct_others,66whproduct_shoes,73logoposition_1,73logoposition_2,73logoposition_3,73logoposition_4,73logoposition_5,73logoposition_6,73logoposition_7,73logoposition_8,73logoposition_9,74logotype_both,74logotype_image,74logotype_text,83productposition_1,83productposition_2,83productposition_3,83productposition_4,83productposition_5,83productposition_6,83productposition_7,83productposition_8,83productposition_9,10angle_dutch,10angle_extreme_high,10angle_eye,10angle_high,10angle_low,10angle_other
0,0,0,0,0,2,0,0,0,0,0,3,1610,933,1614,3,34,37,0,0,0,452,452,453,452,0,1,1571,1761,1571,1571,0,1441,1761,1441,1441,1441,1511,0,1058,1760,1057,1057,1057,1056,0,0,1761,0,1761,0,1761,0,0,0,0,0,0,0,0,0,0,39,66,37,0,0,0,0,452,452,452,452,452,452,452,452,452,0,0,0,0,1571,1571,1571,1571,1571,1571,1571,1571,1571,1441,1441,1441,1441,1441,1441,1441,1441,1441,1511,1511,1511,1511,1511,1511,1511,1511,1057,1057,1057,1057,1057,1057,1057,1057,1057,1057,1057,1057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
col = '8product'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

df_product = df_sns_clean[(df_sns_clean['8product'] == 1)]

col = '81productexpose'
result = chi_sns(df_product,col,{1:'Whole',0:'Part'})
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)


[8product]
          8product            
                 N     Y   All
1SNS                          
Facebook       152   848  1000
instagram       84   913   997
All            236  1761  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   21.9880
1                    p-value =    0.0000
2               Cramer's phi =    0.1049 

Importance: ***

[81productexpose]
          81productexpose            
                     Part Whole   All
1SNS                                 
Facebook              154   694   848
instagram             207   706   913
All                   361  1400  1761 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    5.4923
1                    p-value =    0.0191
2               Cramer's phi =    0.0558 

Importance: *



In [44]:
position_col_name = '83productposition_'
df_position = df_product

print("[[%s]]\n\n"%(position_col_name.replace("_","")))
position_col = [position_col_name in s for s  in df_position.columns]
position_columns = df_model.loc[:,position_col].columns

for col in position_columns:
    print('[%s]'%col)
    result = chi_sns(df_model,col)
    print(result[0],'\n')
    print(result[1],'\n')
    p = result[1].iloc[1,1]
    score = importance(p)
    print('Importance: %s\n'%score)


[[83productposition]]


[83productposition_1]
          83productposition_1           
                            N    Y   All
1SNS                                    
Facebook                  371  288   659
instagram                 399  250   649
All                       770  538  1308 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    3.6260
1                    p-value =    0.0569
2               Cramer's phi =    0.0527 

Importance: X

[83productposition_2]
          83productposition_2           
                            N    Y   All
1SNS                                    
Facebook                  200  459   659
instagram                 222  427   649
All                       422  886  1308 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.2264
1                    p-value =    0.1357
2               Cramer's phi =    0.0413 

Importance: X

[83productposition_3]
          83productposition_3           
           

### Camera Angle

In [42]:
angle = {1:'extreme high',2:'high',3:'eye',4:'low',5:'dutch',6:'other'}
chi_iter_sns(df_sns_clean,'10angle',angle)

[dutch]
          dutch         
              0  1   All
1SNS                    
Facebook   1000  0  1000
instagram   996  1   997
All        1996  1  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    1.0035
1                    p-value =    0.3165
2               Cramer's phi =    0.0224 

Importance: X

[extreme high]
          extreme high          
                     0   1   All
1SNS                            
Facebook           982  18  1000
instagram          973  24   997
All               1955  42  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.8941
1                    p-value =    0.3444
2               Cramer's phi =    0.0212 

Importance: X

[eye]
           eye            
             0     1   All
1SNS                      
Facebook   259   741  1000
instagram  250   747   997
All        509  1488  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    0.1788
1          

### Hash Tag

In [45]:
col = '12hashtag'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[12hashtag]
          12hashtag            
                  N     Y   All
1SNS                           
Facebook        753   247  1000
instagram       239   758   997
All             992  1005  1997 

                Chi-square test   results
0  Pearson Chi-square ( 1.0) =   526.1452
1                    p-value =     0.0000
2               Cramer's phi =     0.5133 

Importance: ***



In [47]:
df_sns_clean['12_2_5'].describe()

count    1997.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: 12_2_5, dtype: float64

In [48]:
df_hashtag = df_sns_clean[df_sns_clean['12hashtag']==1]

In [49]:
print('[[Total]]\n')

ttest_col = ['12_1number','12_2_1content', '12_2_2', '12_2_3', '12_2_4', '12_2_6']


for col in ttest_col:
    print('[%s]'%col)
    result = t_test_sns(df_sns_clean,col)
    p = result[1].iloc[3,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)
    
print('[[When Hashtag]]\n')

ttest_col = ['12_1number','12_2_1content', '12_2_2', '12_2_3', '12_2_4', '12_2_6']


for col in ttest_col:
    print('[%s]'%col)
    result = t_test_sns(df_hashtag,col)
    p = result[1].iloc[3,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)
    

[[Total]]

[12_1number]
    Variable       N      Mean        SD        SE  95% Conf.  Interval
0   Facebook  1000.0  0.292000  0.590834  0.018684   0.255336  0.328664
1  Instagram   997.0  1.348044  1.396434  0.044226   1.261258  1.434830
2   combined  1997.0  0.819229  1.194456  0.026729   0.766809  0.871648 

                                Independent t-test    results
0             Difference (Facebook - Instagram) =     -1.0560
1                            Degrees of freedom =   1995.0000
2                                             t =    -22.0193
3                         Two side test p value =      0.0000
4  Mean of Facebook > mean of Instagram p value =      0.0000
5  Mean of Facebook < mean of Instagram p value =      1.0000
6                                     Cohen's d =     -0.9855
7                                     Hedge's g =     -0.9851
8                                 Glass's delta =     -1.7874
9                                             r =      0.4422 

Im

## Text

In [50]:
pd.DataFrame(df_sns_clean.isna().sum()).T

,BrandID,BrandCategory,PhotoID,PostURL,PhotoURL,PostDate,PostTime,DownloadDate,Type,type_video,Like,LikeAlbum,Comments,Comments_album,NumHashtags,Content,ContentWithoutLink,1SNS,2brand,3model,31number,32face,33body,34modelposition,4background,5text,51words,52,53textposition,54promotingmessa,6brandname,61brandexpose,62,63brandposition,64brandvisib,65partofprod,66whproduct,7logo,71logoexposure,72,73logoposition,74logotype,75logovisib,76logopartofprod,8product,81productexpose,82,83productposition,9,10angle,11,12hashtag,12_1number,12_2_1content,12_2_2,12_2_3,12_2_4,12_2_5,12_2_6,13text,13_1,13_2promo,13_2promo_wo_kor,13_3link,BrandCategory_SPA,BrandCategory_casual,BrandCategory_luxuries,BrandCategory_sport,34modelposition_1,34modelposition_2,34modelposition_3,34modelposition_4,34modelposition_5,34modelposition_6,34modelposition_7,34modelposition_8,34modelposition_9,4background_indoor,4background_others,4background_outdoor,4background_wall,53textposition_1,53textposition_2,53textposition_3,53textposition_4,53textposition_5,53textposition_6,53textposition_7,53textposition_8,53textposition_9,63brandposition_1,63brandposition_2,63brandposition_3,63brandposition_4,63brandposition_5,63brandposition_6,63brandposition_7,63brandposition_8,63brandposition_9,66whproduct_accessories,66whproduct_bag,66whproduct_clothes_bottom,66whproduct_clothes_top,66whproduct_cosmetics,66whproduct_hat,66whproduct_others,66whproduct_shoes,73logoposition_1,73logoposition_2,73logoposition_3,73logoposition_4,73logoposition_5,73logoposition_6,73logoposition_7,73logoposition_8,73logoposition_9,74logotype_both,74logotype_image,74logotype_text,83productposition_1,83productposition_2,83productposition_3,83productposition_4,83productposition_5,83productposition_6,83productposition_7,83productposition_8,83productposition_9,10angle_dutch,10angle_extreme_high,10angle_eye,10angle_high,10angle_low,10angle_other
0,0,0,0,0,3,0,0,0,0,0,4,1836,1019,1841,5,37,40,0,0,0,648,648,649,648,0,1,1722,1997,1722,1722,0,1623,1997,1623,1623,1623,1747,0,1215,1996,1214,1214,1214,1213,0,236,1997,236,1997,0,1997,0,0,0,0,0,0,0,0,0,0,42,70,40,0,0,0,0,648,648,648,648,648,648,648,648,648,0,0,0,0,1722,1722,1722,1722,1722,1722,1722,1722,1722,1623,1623,1623,1623,1623,1623,1623,1623,1623,1747,1747,1747,1747,1747,1747,1747,1747,1214,1214,1214,1214,1214,1214,1214,1214,1214,1214,1214,1214,236,236,236,236,236,236,236,236,236,0,0,0,0,0,0


In [51]:
len(df_sns_clean['ContentWithoutLink'][0])

158

In [52]:
col = '13text'

result = chi_sns(df_sns_clean,col)
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[13text]
          13text            
               N     Y   All
1SNS                        
Facebook      42   958  1000
instagram      0   997   997
All           42  1955  1997 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   42.7736
1                    p-value =    0.0000
2               Cramer's phi =    0.1464 

Importance: ***



In [53]:
ttest_col = ['13_1']


for col in ttest_col:
    print('[%s]'%col)
    result = t_test_sns(df_sns_clean,col)
    p = result[1].iloc[3,1]
    print(result[0],'\n')
    print(result[1],'\n')
    score = importance(p)
    print('Importance: %s\n'%score)

[13_1]
    Variable       N       Mean         SD        SE  95% Conf.   Interval
0   Facebook  1000.0  21.880000  19.088792  0.603641  20.695451  23.064549
1  Instagram   997.0  22.413240  15.554405  0.492613  21.446561  23.379918
2   combined  1997.0  22.146219  17.411835  0.389633  21.382090  22.910349 

                                Independent t-test    results
0             Difference (Facebook - Instagram) =     -0.5332
1                            Degrees of freedom =   1995.0000
2                                             t =     -0.6842
3                         Two side test p value =      0.4939
4  Mean of Facebook > mean of Instagram p value =      0.2470
5  Mean of Facebook < mean of Instagram p value =      0.7530
6                                     Cohen's d =     -0.0306
7                                     Hedge's g =     -0.0306
8                                 Glass's delta =     -0.0279
9                                             r =      0.0153 

Importa

In [54]:
df_posting = df_sns_clean[(df_sns_clean['13text']==1)&(df_sns_clean['13_2promo']!=3)]
result = chi_sns(df_posting,'13_2promo')
p = result[1].iloc[1,1]
score = importance(p)

print('[%s]'%col)
print(result[0],'\n')
print(result[1],'\n')
print('Importance: %s\n'%score)

[13_1]
          13_2promo           
                2.0    Y   All
1SNS                          
Facebook        532  398   930
instagram       422  575   997
All             954  973  1927 

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =   42.6038
1                    p-value =    0.0000
2               Cramer's phi =    0.1487 

Importance: ***

